In [1]:
-- Code for Wide Residual Networks http://arxiv.org/abs/1605.07146
-- (c) Sergey Zagoruyko, 2016
require 'xlua'
require 'optim'
require 'image'
require 'cunn'
require 'cudnn'
c = require 'trepl.colorize'
json = require 'cjson'
require'augmentation'
model_utils = require 'model_utils'   ---sjmod
require'provider'
require'nngraph'
require 'csvigo'

-- for memory optimizations and graph generation
local optnet = require 'optnet'
local graphgen = require 'optnet.graphgen'
local iterm = require 'iterm'
require 'iterm.dot'

opt = {
  save = 'logs_testingvisu/resnet164_att2',
  batchSize = 64, --128,
  learningRate = 0.1,   ------overwritten
  learningRateDecay = 0,   ------overwritten
  learningRateDecayRatio = 0.2,   ------overwritten
  weightDecay = 0.0005,
  dampening = 0,
  momentum = 0.9,
  epoch_step = "80",   ------overwritten
  max_epoch = 300,   ------overwritten

  model_1_1 = 'model_1_1.t7',
  model_1_2 = 'model_1_2.t7',
  model_1_3 = 'model_1_3.t7',

  model_2_3 = 'model_2_3.t7',
  
  model_3_2 = 'model_3_2.t7',
  model_3_3 = 'model_3_3.t7',

  model_4 = 'model_4.t7',

  optimMethod = 'sgd',
  init_value = 10,
  depth = 50,
  shortcutType = 'A',
  nesterov = false,   ------overwritten
  dropout = 0,
  hflip = true,
  randomcrop = 4,
  imageSize = 32,
  randomcrop_type = 'zero',   ------overwritten
  cudnn_fastest = true,
  cudnn_deterministic = false,
  optnet_optimize = true,
  generate_graph = false,
  multiply_input_factor = 1,
  widen_factor = 1,
}
opt = xlua.envparams(opt)

opt.epoch_step = tonumber(opt.epoch_step) or loadstring('return '..opt.epoch_step)()
print(opt)


{
  optnet_optimize : true
  hflip : true
  randomcrop : 4
  batchSize : 64
  model_1_1 : model_1_1.t7
  nesterov : false
  cudnn_deterministic : false
  multiply_input_factor : 1
  momentum : 0.9
  optimMethod : sgd
  dropout : 0
  weightDecay : 0.0005
  generate_graph : false
  init_value : 10
  widen_factor : 1
  imageSize : 32
  model_3_3 : model_3_3.t7
  learningRateDecayRatio : 0.2
  dampening : 0
  shortcutType : A
  model_4 : model_4.t7
  depth : 50
  model_1_3 : model_1_3.t7
  model_2_3 : model_2_3.t7
  model_1_2 : model_1_2.t7
  randomcrop_type : zero
  learningRateDecay : 0
  cudnn_fastest : true
  model_3_2 : model_3_2.t7
  max_epoch : 300
  save : logs_testingvisu/resnet164_att2


In [2]:
print(c.blue '==>' ..' loading data')
--provider= torch.load('/media/sjvision/DATASETDISK/cifars/Cifar10/cifar10_whitened_orig.t7')

--provider = torch.load('/media/sjvision/DATASETDISK/atest/Event8/t7_files/event8_whitened.t7')  ---sjmod
--provider = torch.load('/media/sjvision/DATASETDISK/atest/Caltech101/t7_files/caltech101_whitened.t7')  ---sjmod
--provider = torch.load('/media/sjvision/DATASETDISK/atest/IndoorSceneRecognition67/t7_files/indoorscenereco67_whitened.t7')  -
provider = torch.load('/media/sjvision/DATASETDISK/atest/StanfordAction40/t7_files/stanfordaction40_whitened.t7')

print(provider.testData.labels[55])
itorch.image(provider.testData.data[55])

  epoch_step : 80
  learningRate : 0.1
}


==> loading data	


[string "print(c.blue '==>' ..' loading data')..."]:9: attempt to index global 'provider' (a nil value)
stack traceback:
	[string "print(c.blue '==>' ..' loading data')..."]:9: in main chunk
	[C]: in function 'xpcall'
	/home/sjvision/torch/install/share/lua/5.1/itorch/main.lua:210: in function </home/sjvision/torch/install/share/lua/5.1/itorch/main.lua:174>
	/home/sjvision/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	.../sjvision/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	.../sjvision/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	.../sjvision/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/home/sjvision/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x00406670: 

In [ ]:
print(c.blue '==>' ..' configuring models')
----------------------------------------------------------------
model_1_1 = nn.Sequential()

model_1_2 = nn.Sequential()
model_2_2 = nn.Sequential()
model_3_2 = nn.Sequential()

model_1_3 = nn.Sequential()
model_2_3 = nn.Sequential()
model_3_3 = nn.Sequential()

model_4 = nn.Sequential()
----------------------------------------------------------------
net_1_1 = torch.load('logs/cifar100_resnet_att2_conv_linear_256/' ..opt.model_1_1):cuda()
net_1_2 = torch.load('logs/cifar100_resnet_att2_conv_linear_256/'..opt.model_1_2):cuda()
net_1_3 = torch.load('logs/cifar100_resnet_att2_conv_linear_256/'..opt.model_1_3):cuda()

net_2_3 = torch.load('logs/cifar100_resnet_att2_conv_linear_256/'..opt.model_2_3):cuda()

net_3_2 = torch.load('logs/cifar100_resnet_att2_conv_linear_256/'..opt.model_3_2):cuda()
net_3_3 = torch.load('logs/cifar100_resnet_att2_conv_linear_256/'..opt.model_3_3):cuda()

net_4 = torch.load('logs/cifar100_resnet_att2_conv_linear_256/'..opt.model_4):cuda()
----------------------------------------------------------------


In [ ]:
do
   ------Main Model----------INITIALIZATION-----------------

   model_1_1:add(nn.Copy('torch.FloatTensor','torch.CudaTensor'):cuda())   
   model_1_1:add(net_1_1) ---- adding the network -- sjmod
   model_1_2:add(net_1_2) ---- adding the network -- sjmod 
   model_1_3:add(net_1_3) ---- adding the network -- sjmod
   
   model_2_3:add(net_2_3) ---- adding the network -- sjmod    
   
   model_3_2:add(net_3_2) ---- adding the network -- sjmod
   model_3_3:add(net_3_3) ---- adding the network -- sjmod

   model_4:add(net_4) ---- adding the network -- sjmod
  
end

In [ ]:

--[[
model_all = {}
table.insert(model_all, model_1_1)

table.insert(model_all, model_1_2)
table.insert(model_all, model_2_2)
table.insert(model_all, model_3_2)

table.insert(model_all, model_1_3)
table.insert(model_all, model_2_3)
table.insert(model_all, model_3_3)

table.insert(model_all, model_4)
local parameters, gradParameters = model_utils.combine_all_parameters(model_all)
--local parameters,gradParameters = model:getParameters()   --sjmod

opt.n_parameters = parameters:numel()
print('Network has ', parameters:numel(), 'parameters')

print(c.blue'==>' ..' setting criterion')
local criterion = nn.CrossEntropyCriterion():cuda()
--]]
function test()
  model_1_1:evaluate()
  model_1_2:evaluate()
  model_1_3:evaluate()
    
  model_2_3:evaluate()
  
  model_3_2:evaluate()
  model_3_3:evaluate()

  model_4:evaluate()

  local confusion = optim.ConfusionMatrix(10)
  nsamples = 600 --provider.testData.data:size(1)
  bs = 1 print(nsamples)
  --fv_output = torch.zeros(nsamples,384)  
    
  for i = 1,nsamples,bs do

   local lfeat_160 = model_1_1:forward(provider.testData.data:narrow(1,i,bs))
   local lfeat_320 = model_1_2:forward(lfeat_160)
   local lfeat_640 = model_1_3:forward(lfeat_320)

   local gfeat_640 = model_2_3:forward(lfeat_640)
   gfeat_640 = nn.Reshape(1,256):cuda():forward(gfeat_640)
        
   local att_con_320 = model_3_2:forward({lfeat_320,gfeat_640})
   local att_con_640 = model_3_3:forward({lfeat_640,gfeat_640})
        
   local pred = model_4:forward({att_con_320[2],att_con_640[2]})
    ------------1. Visualizations 
    csvigo.save(string.format('/media/sjvision/DATASETDISK/atest/StanfordAction40/attmaps/attmaps_resnet_100/%03d_%s',i,'level2.txt'),att_con_320[1]:squeeze():double():totable())
    csvigo.save(string.format('/media/sjvision/DATASETDISK/atest/StanfordAction40/attmaps/attmaps_resnet_100/%03d_%s',i,'level3.txt'),att_con_640[1]:squeeze():double():totable())
    --[[
    itorch.image(image.scale(att_con_320[1]:float(),32,32))
    itorch.image(image.scale(att_con_640[1]:float(),32,32))
    itorch.image(provider.testData.data:narrow(1,i,bs))
    --]]
   -------------2. Save the features to a file
   --fv_output[i] =  model_4:get(1).modules[9].output:squeeze():float()
   --print(ind)
   --print(provider.testData.labels[i])   
        --[[
        confusion:batchAdd(pred, provider.testData.labels:narrow(1,i,bs))   -----testing fwd   ----sjmod
        confusion:updateValids()
        if confusion.totalValid ~= 0 then
            image.save(i ..'right_atten1.png', image.scale(att_con_320[1]:float(),32,32))
            image.save(i ..'right_atten2.png', image.scale(att_con_640[1]:float(),32,32))
            image.save(i ..'right.png',provider.testData.data:narrow(1,i,bs):squeeze())
        else
            image.save(i ..'wrong_atten1.png', image.scale(att_con_320[1]:float(),32,32))
            image.save(i ..'wrong_atten2.png', image.scale(att_con_640[1]:float(),32,32))
            image.save(i ..'wrong.png',provider.testData.data:narrow(1,i,bs):squeeze())
        end 
       confusion:zero()
        ]]--
  end
    --fv_output = fv_output:totable()
    --csvigo.save('logs_objectdiscoverydata/resnet164_att2_zca_100.txt',fv_output)
    return 0 --confusion.totalValid * 100
end


In [ ]:
for epoch=1,1 do
  local test_acc, test_time =test()  ----sjmod
  print(test_acc)
end